In [3]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import numpy as np

# FUNCTION TO COMPUTE DFF
import os
from utility_classification import sum_pixels_in_registered_mask, fix_trials



Autosaving every 180 seconds


In [4]:
import string, re
from scipy.signal import butter, filtfilt, cheby1

#select code 04/02/07 triggers;
def get_04_triggers_with_lockout(root_dir, recording, lockout_window=10):
    
    # make sure locs
    try:
        locs_44threshold = np.load(root_dir + '/tif_files/' + recording + '/' + recording + '_locs44threshold.npy')
    except:
        print ("locs 44 thrshold missing", recording)
        return np.zeros((0),'float32'), np.zeros((0),'float32')
        
    codes = np.load(root_dir + '/tif_files/' + recording + '/'+recording + '_code44threshold.npy')
    code = b'04'
    idx = np.where(codes==code)[0]
    locs_selected = locs_44threshold[idx]

    if locs_selected.shape[0]==0:
        return np.zeros((0),'float32'), np.zeros((0),'float32')

    diffs = locs_selected[1:]-locs_selected[:-1]
    idx = np.where(diffs>lockout_window)[0]
    
    locs_selected_with_lockout = locs_selected[idx+1] 
    if locs_selected_with_lockout.shape[0]==0:
        return np.zeros((0),'float32'), np.zeros((0),'float32')

    # ADD FIRST VAL
    if locs_selected[0]>lockout_window:
        locs_selected_with_lockout = np.concatenate(([locs_selected[0]], locs_selected_with_lockout), axis=0)

    return locs_selected, locs_selected_with_lockout
    

def find_nearest(array, value):
    return (np.abs(array-value)).argmin()

def load_reclength(filename):
    """ Load realtime length of a single session. Probably should be in session, but was quicker to dump here"""

    #print ("FILENAME: ", filename)
    text_file = open(filename, "r")
    lines = text_file.read().splitlines()
    event_text = []
    for line in lines:
        event_text.append(re.split(r'\t+',line))

    #Delete false starts from event file
    for k in range(len(event_text)-1,-1,-1):        #Search backwards for the 1st occurence of "date" indicating last imaging start
                                                    #NB: There can be multiple false starts WHICH DON"T LINE UP - NEED TO IGNORE SUCH SESSIONS
        if event_text[k][0]=='date': 
            event_text = event_text[k+2:]         #Remove first 2 lines
            break

    if len(event_text)==0:
        reclength = 0
    else:
        if event_text[-1][2] != "None": 
            reclength = 0
        else: 
            reclength = float(event_text[-1][3])

    return reclength
    

# FUNCTION TO COMPUTE DFF
def compute_DFF_function(
                        root_dir,
                        dff_method, # 'globalAverage' or 'slidingWindow'
                        recording,
                        locs_selected,
                        n_sec_window = 10
                        ):

    
    # ###################################################
    # ###################################################
    # ###################################################
    # SET DEFAULT PARAMETERS
    #n_sec_window = 10
    low_cut = 0.1
    high_cut = 6.0
    img_rate = np.loadtxt(root_dir+'/img_rate.txt')
    selected_dff_filter = 'butterworth'

    # MAKE FILENAMES
    tif_files = root_dir+'/tif_files.npy'
    event_files = root_dir + '/event_files.npy'
    aligned_fname = root_dir + '/tif_files/'+recording + '/'+recording + "_aligned.npy"
    #print ("aligned fname: ", aligned_fname)

    rec_filename = root_dir + '/tif_files/'+recording + '/'+recording +'.tif'
    #print ("rec_fileame;", rec_filename)
    n_sec = float(n_sec_window)
        
    # Load aligned/filtered data and find ON/OFF light;
    #images_file = self.parent.animal.home_dir+self.parent.animal.name+'/tif_files/'+self.rec_filename+'/'+self.rec_filename+'_aligned.npy'
    images_file = aligned_fname
    try:
        aligned_images = np.load(images_file)
    except:
        print ("missing aligned images - skipping session", recording)
        return np.zeros((0),'float32')

    
    # Find blue light on/off 
    blue_light_threshold = 400  #Intensity threshold; when this value is reached - imaging light was turned on
    start_blue = 0; end_blue = len(aligned_images)
    
    if np.average(aligned_images[0])> blue_light_threshold:    #Case #1: imaging starts with light on; need to remove end chunk; though likely bad recording
        for k in range(len(aligned_images)):
            if np.average(aligned_images[k])< blue_light_threshold:
                #self.aligned_images = self.aligned_images[k:]
                end_blue = k
                break
    else:                                                           #Case #2: start with light off; remove starting and end chunks;
        #Find first light on
        for k in range(len(aligned_images)):
            if np.average(aligned_images[k])> blue_light_threshold:
                start_blue = k
                break

        #Find light off - count backwards from end of imaging data
        for k in range(len(aligned_images)-1,0,-1):
            if np.average(aligned_images[k])> blue_light_threshold:
                end_blue= k
                break
                
                
    #self.lowcut = float(self.parent.filter_low.text())
    #self.highcut = float(self.parent.filter_high.text())
        
    #if self.selected_dff_filter == 'nofilter':
    #    pass; #already loaded nonfiltered self.aligned_images above
    #else:
    filtered_filename = images_file[:-4]+'_'+selected_dff_filter+'_'+str(low_cut)+'hz_'+str(high_cut)+'hz.npy'
    if os.path.exists(filtered_filename):
        try:
            aligned_images = np.load(filtered_filename, allow_pickle=True)
        except:
            print ("aligned filtered images corrupt... recomputing: ", filtered_filename)
            filter_data(root_dir, recording)
            aligned_images = np.load(filtered_filename)
    else:
        print ("aligned filtered images missing... recomputing: ", filtered_filename)
        filter_data(root_dir, recording)
        aligned_images = np.load(filtered_filename)
        
    aligned_images = aligned_images[start_blue:end_blue]
    
    # compute # of images in stack
    n_images=len(aligned_images)


    # Determine if imaging rate correct 
    temp_tif_files = np.load(tif_files)
    temp_event_files = np.load(event_files)
    if len(temp_event_files)==1:
        temp_event_files = temp_event_files[0]
    #print ("temp_tif files;l", temp_tif_files)
    #print ("rec_filename: ", rec_filename)

    index = None
    for k in range(len(temp_tif_files)):
        try:
            temp_temp = temp_tif_files[k].decode("utf-8").replace('12TB/in_vivo/tim','4TBSSD').replace(
                                                    '10TB/in_vivo/tim','4TBSSD')#.replace("b'/", "'/")
        except:
            temp_temp = temp_tif_files[k].replace('12TB/in_vivo/tim','4TBSSD').replace(
                                                    '10TB/in_vivo/tim','4TBSSD')#.replace("b'/", "'/")
        if rec_filename in temp_temp:
            index = k 
            break
    
    if index == None:
        
        print ("DID NOT FIND MATCH between imaging and lever ---- RETURNING ")
        #print (temp_tif_files)
        return np.zeros((0),'float32')
    
    #print ("INDEX: ", index)
    #print ("temp event files indexed: ", len(temp_event_files[index]))
    # load the reclength based 
    try:
        reclength = load_reclength(temp_event_files[index].decode("utf-8").replace(
                                                    '10TB/in_vivo/tim','4TBSSD'))
    except:
        reclength = load_reclength(temp_event_files[index].replace(
                                                    '10TB/in_vivo/tim','4TBSSD'))

    if reclength ==0:
        print ("zero length recording exiting (excitation light failure)", recording)
        return np.zeros((0),'float32')
    
    # compute imaging rate; 
    session_img_rate = n_images/reclength

    if abs(session_img_rate-float(img_rate))<0.01:         #Compare computed session img_rate w. experimentally set img_rate
        np.save(images_file.replace('_aligned.npy','')+'_img_rate', session_img_rate)
    else:
        np.save(images_file.replace('_aligned.npy','')+'_img_rate', session_img_rate)
        print ("Imaging rates between aligned and session are incorrect, exiting: ", session_img_rate)
        return np.zeros((0),'float32')


    # Find times of triggers from lever pull threshold times
    trigger_times = locs_selected
    frame_times = np.linspace(0, reclength, n_images)             #Divide up reclength in number of images
    img_frame_triggers = []
    for i in range(len(trigger_times)):
        #img_frame_triggers.append(self.find_previous(frame_times, trigger_times[i])) 
        img_frame_triggers.append(find_nearest(frame_times, trigger_times[i]))     #Two different functions possible here; 
    
    #BASELINE FOR GLOBAL BASELINE REMOVAL
    mean_file = root_dir + '/tif_files/'+recording + '/'+recording + '_aligned_mean.npy'
    if os.path.exists(mean_file)==False:
        aligned_fname = root_dir + '/tif_files/'+recording + '/'+recording + "_aligned.npy"
        images_file = aligned_fname
        images_aligned = np.load(images_file)
        images_aligned_mean = np.mean(images_aligned, axis=0)
        np.save(images_file[:-4]+'_mean', images_aligned_mean)

    global_mean = np.load(mean_file)

    abstimes = np.load(root_dir + '/tif_files/'+recording + '/'+recording + '_abstimes.npy')
    abspositions = np.load(root_dir + '/tif_files/'+recording + '/'+recording + '_abspositions.npy')

    data_stm = []; traces = []; locs = []; codes = []
    counter=-1
    window = n_sec * session_img_rate      #THIS MAY NOT BE GOOD ENOUGH; SHOULD ALWAYS GO BACK AT LEAST X SECONDS EVEN IF WINDOW IS ONLY 1SEC or 0.5sec...
                                                            #Alternatively: always compute using at least 3sec window, and then just zoom in
    for trigger in img_frame_triggers:
        counter+=1
        #NB: Ensure enough space for the sliding window; usually 2 x #frames in window
        if trigger < (2*window) or trigger>(n_images-window): 
            continue  #Skip if too close to start/end

        #add locs and codes
        #locs.append(locs_44threshold_selected[counter])
        #codes.append(code_44threshold_selected[counter])

        # load data chunk working with
        data_chunk = aligned_images[int(trigger-window):int(trigger+window)]

        if dff_method == 'globalAverage':
            data_stm.append((data_chunk-global_mean)/global_mean)    #Only need to divide by global mean as original data_chunk did not have mean img added in
            
        elif dff_method == 'slidingWindow':            #Use baseline -2*window .. -window
            baseline = np.average(aligned_images[int(trigger-2*window):int(trigger-window)], axis=0)
            data_stm.append((data_chunk-baseline)/baseline)
        
        #***PROCESS TRACES - WORKING IN DIFFERENT TIME SCALE
        lever_window = int(120*n_sec)    #NB: Lever window is computing in real time steps @ ~120Hz; and discontinuous;
        t = np.linspace(-lever_window*0.0082,
                        lever_window*0.0082, 
                        lever_window*2)
        #lever_position_index = find_nearest(np.array(self.abstimes), self.locs_44threshold[counter])
        lever_position_index = find_nearest(np.array(abstimes), locs_selected[counter])
        
        lever_trace = abspositions[int(lever_position_index-lever_window):int(lever_position_index+lever_window)]

        if len(lever_trace)!=len(t):    #Extraplote missing data
            lever_trace = np.zeros(lever_window*2,dtype=np.float32)
            for k in range(-lever_window,lever_window,1):
                lever_trace[k+lever_window] = self.abspositions[k+lever_window]     #Double check this...

        traces.append(lever_trace)

    data_stm = np.array(data_stm)
    
    return data_stm


def filter_data(root_dir,
                recording,
                ):

    
    # ###################################################
    # ###################################################
    # ###################################################
    # SET DEFAULT PARAMETERS
    #n_sec_window = 10
    low_cut = 0.1
    high_cut = 6.0
    img_rate = 30.0
    selected_dff_filter = 'butterworth'

    # MAKE FILENAMES
    generic_mask_fname = root_dir + '/genericmask.txt'
    tif_files = root_dir+'tif_files.npy'
    event_files = root_dir + 'event_files.npy'
    aligned_fname = root_dir + '/tif_files/'+recording + '/'+recording + "_aligned.npy"
    
    #print ("FILTERING DATA: ", aligned_fname)
    
    # FILTERING STEP
    images_file = aligned_fname
    
    filter_type = selected_dff_filter
    lowcut = low_cut
    highcut = high_cut
    fs = img_rate

    #Check to see if data requested exists- THIS CHECK WAS ALREADY DONE PRIOR TO ENTERING FUNCTION
    if False:
        if os.path.exists(images_file[:-4]+'_'+filter_type+'_'+str(lowcut)+'hz_'+str(highcut)+'hz.npy'):
            #print ("filtered data already exists...")
            return

    #Load aligned images
    if os.path.exists(images_file):
        images_aligned = np.load(images_file)
    else:
        print (" ...missing aligned images... NEED TO RUN ALIGN ALGORITHMS", images_file)
        return None
        
        # TODO IMPLMENET ALIGNMENT TOOL
        #images_aligned = align_images2(self)
        
    #Save mean of images_aligned if not already done
    if os.path.exists(images_file[:-4]+'_mean.npy')==False: 
        images_aligned_mean = np.mean(images_aligned, axis=0)
        np.save(images_file[:-4]+'_mean', images_aligned_mean)
    else:
        images_aligned_mean = np.load(images_file[:-4]+'_mean.npy')
            
    #Load mask - filter only datapoints inside mask
    n_pixels = len(images_aligned[0])
    generic_coords = np.loadtxt(generic_mask_fname)
    generic_mask_indexes=np.zeros((n_pixels,n_pixels))
    for i in range(len(generic_coords)): generic_mask_indexes[int(generic_coords[i][0])][int(generic_coords[i][1])] = True

    #Filter selection and parameters
    if filter_type == 'butterworth':
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        order = 2
        b, a = butter(order, [low, high], btype='band')
    elif filter_type == 'chebyshev':
        nyq = fs / 2.0
        order = 4
        rp = 0.1
        Wn = [lowcut / nyq, highcut / nyq]
        b, a = cheby1(order, rp, Wn, 'bandpass', analog=False)
    
    
    #Load individual pixel time courses; SWITCH TO UNRAVEL HERE****
    import time
   
    filtered_array = np.zeros(images_aligned.shape, dtype=np.float16)
    now = time.time(); start_time = now
    cutoff=n_pixels
    #from tqdm import tqdm
    #for p1 in tqdm(range(n_pixels)):
    for p1 in range(n_pixels):
        now=time.time(); n_pixels_in=0
        for p2 in range(n_pixels):
            if generic_mask_indexes[p1,p2]==False:
                filtered_array[:,p1,p2] = np.float16(filtfilt(b, a, images_aligned[:,p1,p2])); n_pixels_in+=1   #filter pixel inside mask
        
    np.save(images_file[:-4]+'_'+filter_type+'_'+str(lowcut)+'hz_'+str(highcut)+'hz', 
            filtered_array+np.float16(images_aligned_mean))

    return 

def compute_trial_courses_ROI(recording, root_dir):

    #try: 
        fname_04 = root_dir + '/tif_files/' + recording + "_data_04_code_trial_timeCourses.npy"
        fname_random = root_dir + '/tif_files/' + recording + "_data_random_code_trial_timeCourses.npy"

        # SET PARAMETERS
        n_sec_window = 10
        dff_method = 'globalAverage'

        if os.path.exists(fname_04)==False:
            # select code 04/02/07 triggers;
            try:
                locs_44threshold = np.load(root_dir + '/tif_files/' + recording + '/' + recording + '_locs44threshold.npy')
            except:
                print ("locs 44 thrshold missing", recording)
                return None
            
            codes = np.load(root_dir + '/tif_files/' + recording + '/'+recording + '_code44threshold.npy')
            code = b'04'
            idx = np.where(codes==code)[0]
            locs_selected = locs_44threshold[idx]

            # CALL FUNCTION;
            data_stm = compute_DFF_function(
                                    root_dir,
                                    dff_method, # 'globalAverage' or 'slidingWindow'
                                    recording,
                                    locs_selected,
                                    n_sec_window
                                    )
            if data_stm is None:
                print ("data_stm is None", recording)
                return


            # CONVERT DATA FROM 128 x 128 to 35 ROIs

            # load Allen Institute afine transformation to scale data
            #maskwarp= np.load('/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Apr22_Week2_30Hz/IA2pm_Apr22_Week2_30Hz_aligned_maskwarp.npy')
            maskwarp = np.load('/home/cat/maskwarp.npy')

            # accumulate mean activity in each ROI
            # input data shape: [# trials, # times, width, height]
            area_ids, trial_courses = sum_pixels_in_registered_mask(data_stm, maskwarp)

            # generate random time corses
            locs_selected = np.float32(np.linspace(30, 1200, data_stm.shape[0]))
            locs_selected = locs_selected + np.random.rand(locs_selected.shape[0])*10-5      

            # CALL FUNCTION;
            data_stm_random = compute_DFF_function(
                                    root_dir,
                                    dff_method, # 'globalAverage' or 'slidingWindow'
                                    recording,
                                    locs_selected,
                                    n_sec_window
                                    )
            if data_stm_random is None:
                return

            # compute random trial time courses
            _, trial_courses_random = sum_pixels_in_registered_mask(data_stm_random, maskwarp)

            #####################################################################
            ######## REMOVE INFINITIES, NANS ETC FROM DATA ######################
            #####################################################################
            if trial_courses.shape[0]==0:
                return
            
            trial_courses_fixed, trial_courses_random_fixed = fix_trials(trial_courses, trial_courses_random)

            np.save(fname_04+ "_area_ids.npy", area_ids)
            np.save(fname_04, trial_courses_fixed)
            #print ("Saved DFF for triggered data: ", fname_04, trial_courses_fixed.shape)

            #offset = DLC_offset
            np.save(fname_random, trial_courses_random_fixed)
            #print ("Saved DFF for random data: ", trial_courses_random_fixed.shape)
    #except:
    #    print ("skipped: ", recording)
    #    pass
   # print ('')


def compute_trial_courses_ROI_any_trigger(recording, 
                                          root_dir,
                                          locs_selected,
                                          feature_name,
                                          n_sec_window=10):


        # GENERATE SAVE FILENAMES 
        fname_04 = root_dir + '/tif_files/' + recording + '/' + recording + "_"+feature_name+"_trial_timeCourses.npy"
        fname_random = root_dir + '/tif_files/' + recording + '/' + recording + "_"+feature_name+"_random_trial_timeCourses.npy"

        # SET PARAMETERS
        #n_sec_window = 10
        dff_method = 'globalAverage'

        if os.path.exists(fname_04)==False:

            # CALL FUNCTION;
            data_stm = compute_DFF_function(
                                    root_dir,
                                    dff_method, # 'globalAverage' or 'slidingWindow'
                                    recording,
                                    locs_selected,
                                    n_sec_window
                                    )
            if data_stm is None:
                print ("data_stm is None", recording)
                return


            # CONVERT DATA FROM 128 x 128 to 35 ROIs

            # load Allen Institute afine transformation to scale data
            #maskwarp= np.load('/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Apr22_Week2_30Hz/IA2pm_Apr22_Week2_30Hz_aligned_maskwarp.npy')
            maskwarp = np.load('/home/cat/maskwarp.npy')

            # accumulate mean activity in each ROI
            # input data shape: [# trials, # times, width, height]
            area_ids, trial_courses = sum_pixels_in_registered_mask(data_stm, maskwarp)

            # generate random time corses
            locs_selected = np.float32(np.linspace(30, 1200, data_stm.shape[0])) 
            locs_selected = locs_selected + np.random.rand(locs_selected.shape[0])*10-5      

            # CALL FUNCTION;
            data_stm_random = compute_DFF_function(
                                    root_dir,
                                    dff_method, # 'globalAverage' or 'slidingWindow'
                                    recording,
                                    locs_selected,
                                    n_sec_window
                                    )
            if data_stm_random is None:
                return

            # compute random trial time courses
            _, trial_courses_random = sum_pixels_in_registered_mask(data_stm_random, maskwarp)

            #####################################################################
            ######## REMOVE INFINITIES, NANS ETC FROM DATA ######################
            #####################################################################
            if trial_courses.shape[0]==0:
                return
            
            trial_courses_fixed, trial_courses_random_fixed = fix_trials(trial_courses, trial_courses_random)

            np.save(fname_04+ "_area_ids.npy", area_ids)
            np.save(fname_04, trial_courses_fixed)
            #print ("Saved DFF for triggered data: ", fname_04, trial_courses_fixed.shape)

            #offset = DLC_offset
            np.save(fname_random, trial_courses_random_fixed)
            #print ("Saved DFF for random data: ", trial_courses_random_fixed.shape)

            

def compute_trial_courses_ROI_code04_trigger(recording, 
                                          root_dir,
                                          feature_name,
                                          lockout_window=10,   # THIS IS THE LOCKOUT WINDOW FOR NO OTHER PULLS
                                          n_sec_window=10,
                                          recompute=False,
                                          midline_filter_flag=False):   # THIS IS THE DFF TIEM COURSE WINDOW; e.g. -10..+10sec


    # SET PARAMETERS
    #n_sec_window = 10
    dff_method = 'globalAverage'

    locs_selected, locs_selected_with_lockout = get_04_triggers_with_lockout(root_dir, 
                                                                            recording,
                                                                            lockout_window)

    # GENERATE SAVE FILENAMES FOR ALL CODE_04 DATA
    fname_04 = (root_dir + '/tif_files/' + recording + '/' + recording + "_"+feature_name+
                "_trial_ROItimeCourses_"+str(n_sec_window)+"sec.npy")

    fname_random = (root_dir + '/tif_files/' + recording + '/' + recording + "_"+feature_name+
                    "_random_trial_ROItimeCourses_"+str(n_sec_window)+"sec.npy")

    # good idea to save these as text to see them after:
    np.savetxt(fname_04[:-4]+"_locs_selected.txt" , locs_selected)

    
    #if os.path.exists(fname_04)==False:
    dff1, dff1_random = generate_arrays_ROI_triggered(root_dir,
                                                         dff_method,
                                                         recording,
                                                         locs_selected,
                                                         n_sec_window,
                                                         fname_04,
                                                         fname_random,
                                                         recompute,
                                                         midline_filter_flag)

    # GENERATE SAVE FILENAMES FOR LOCKOUT DATA
    fname_04 = (root_dir + '/tif_files/' + recording + '/' + recording + "_"+feature_name+
                "_lockout_"+str(lockout_window)+"sec_trial_ROItimeCourses_"+str(n_sec_window)+"sec.npy")
    fname_random = (root_dir + '/tif_files/' + recording + '/' + recording + "_"+feature_name+
                    "_lockout_"+str(lockout_window)+"sec_random_trial_timeCourses_"+str(n_sec_window)+"sec.npy")

    #if os.path.exists(fname_04)==False:
    dff2, dff2_random = generate_arrays_ROI_triggered(root_dir,
                                                         dff_method,
                                                         recording,
                                                         locs_selected_with_lockout,
                                                         n_sec_window,
                                                         fname_04,
                                                         fname_random,
                                                         recompute,
                                                         midline_filter_flag)


    return dff1, dff1_random, dff2, dff2_random



def load_trial_courses_ROI_code04_trigger(recording, 
                                          root_dir,
                                          feature_name,
                                          lockout_window,   # THIS IS THE LOCKOUT WINDOW FOR NO OTHER PULLS
                                          n_sec_window):   # THIS IS THE DFF TIEM COURSE WINDOW; e.g. -10..+10sec


    # GENERATE SAVE FILENAMES FOR ALL CODE_04 DATA
    fname_04 = (root_dir + '/tif_files/' + recording + '/' + recording + "_"+feature_name+
                "_trial_ROItimeCourses_"+str(n_sec_window)+"sec.npy")

    fname_random = (root_dir + '/tif_files/' + recording + '/' + recording + "_"+feature_name+
                    "_random_trial_ROItimeCourses_"+str(n_sec_window)+"sec.npy")

    data_04 = np.load(fname_04)
    data_04_random = np.load(fname_random)

        
    # GENERATE SAVE FILENAMES FOR LOCKOUT DATA
    fname_04 = (root_dir + '/tif_files/' + recording + '/' + recording + "_"+feature_name+
                "_lockout_"+str(lockout_window)+"sec_trial_ROItimeCourses_"+str(n_sec_window)+"sec.npy")
    fname_random = (root_dir + '/tif_files/' + recording + '/' + recording + "_"+feature_name+
                    "_lockout_"+str(lockout_window)+"sec_random_trial_timeCourses_"+str(n_sec_window)+"sec.npy")

    
    data_04_lockout = np.load(fname_04)
    data_04_lockout_random = np.load(fname_random)
    
    
    return data_04, data_04_random, data_04_lockout, data_04_lockout_random


def compute_midline_filter(root_dir,
                              data_stm):
    import yaml

    with open(os.path.join(root_dir,"gcamp.txt"), 'r') as f:
        valuesYaml = yaml.load(f, Loader=yaml.FullLoader)

    val999 = valuesYaml['val999']
    width = valuesYaml['width']
    power = valuesYaml['power']
    maxval = 10 # is not being used currently

    # generate midline filter using t=0 to approximately t=0.5sec brain activity which most likely to generate midline blood pressure 
    # artifacts
    #print ("data stm: ", data_stm.shape)
    midline_filter = motion_mask_parallel(data_stm.mean(0)[data_stm.shape[0]//2: 
                                                           data_stm.shape[0]//2+15].mean(0), 
                                          maxval,  # this value is not used
                                          val999, 
                                          width, 
                                          power)

    return midline_filter


def correct_midline_artifact(data_stm,
                            midline_filter):

    # loop over all frames in data stack and filter out midline activity
    for k in range(data_stm.shape[0]):
        for p in range(data_stm.shape[1]):
            data_stm[k,p] *=midline_filter

    return data_stm 


def generate_arrays_ROI_triggered(root_dir,
                                 dff_method,
                                 recording,
                                 locs_selected,
                                 n_sec_window,
                                 fname_04,
                                 fname_random, 
                                 recompute,
                                 midline_filter_flag):
     
    from tqdm import trange
    
    fname_04_data_stm = fname_04[:-4]+"_all_brain.npy"

    if os.path.exists(fname_04)==False or recompute==True:
        # CALL FUNCTION;
        data_stm = compute_DFF_function(
                                root_dir,
                                dff_method, # 'globalAverage' or 'slidingWindow'
                                recording,
                                locs_selected,
                                n_sec_window
                                )
        if data_stm.shape[0]==0:
            print ("data_stm is None", recording)
            return np.zeros((0), 'float32'), np.zeros((0), 'float32') 
        
        # SAVE data_stm stack
        if False:
            np.save(fname_04_data_stm, data_stm)
        
        # 
        if midline_filter_flag:
            midline_filter = compute_midline_filter(root_dir,
                                              data_stm)
            
            data_stm = correct_midline_artifact(data_stm, 
                                                midline_filter)
            
        # CONVERT DATA FROM 128 x 128 to 35 ROIs

        # load Allen Institute afine transformation to scale data
        maskwarp = np.load('/home/cat/maskwarp.npy')

        # accumulate mean activity in each ROI
        # input data shape: [# trials, # times, width, height]
        area_ids, trial_courses = sum_pixels_in_registered_mask(data_stm, maskwarp)

        
        ########################################################
        ########################################################
        ########################################################
        # generate random time corses
        locs_selected = np.float32(np.linspace(30, 1200, data_stm.shape[0])) 
        locs_selected = locs_selected + np.random.rand(locs_selected.shape[0])*10-5      
        data_stm = None

        
        # CALL FUNCTION;
        data_stm_random = compute_DFF_function(
                                root_dir,
                                dff_method, # 'globalAverage' or 'slidingWindow'
                                recording,
                                locs_selected,
                                n_sec_window
                                )
        
        if data_stm_random is None:
            return np.zeros((0), 'float32'), np.zeros((0), 'float32') 

        # use the same filter as in the event triggered neural activity
        if midline_filter_flag:
           
            data_stm_random = correct_midline_artifact(data_stm_random,
                                                      midline_filter)
            
            
            
        # compute random trial time courses
        _, trial_courses_random = sum_pixels_in_registered_mask(data_stm_random, maskwarp)
        data_stm_random = None
        
        #####################################################################
        ######## REMOVE INFINITIES, NANS ETC FROM DATA ######################
        #####################################################################
        if trial_courses.shape[0]==0 or trial_courses_random.shape[0]==0:
            return np.zeros((0), 'float32'), np.zeros((0), 'float32') 

        #print (trial_courses.shape, trial_courses_random.shape)
        trial_courses_fixed, trial_courses_random_fixed = fix_trials(trial_courses, trial_courses_random)

        np.save(fname_04[:-4]+ "_area_ids.npy", area_ids)
        np.save(fname_04, trial_courses_fixed)
        #print ("Saved DFF for triggered data: ", fname_04, trial_courses_fixed.shape)

        #offset = DLC_offset
        np.save(fname_random, trial_courses_random_fixed)
        #print ("Saved DFF for random data: ", trial_courses_random_fixed.shape)
    
    else:
        
        trial_courses_fixed = np.load(fname_04)

        trial_courses_random_fixed = np.load(fname_random)

        
    return trial_courses_fixed, trial_courses_random_fixed



def sigmoid_function(x, a, b):

    return np.clip(a*(np.ma.log(x) - np.ma.log(1 - x))+b, 0, 1)      #Compute sigmoid and cut off values below 0 and above 1
    
    
def mangle(width, x, img_temp, maxval, power, val999):
    
    mu = 0 #Select approximate midline as centre of gaussian
    sig = width
    
    a = .005       #The steepness of the sigmoid function
    b = val999        #% of maxval to cutoff


    #Normalize img_temp for sigmoid to work properly
    #img_temp_norm = (img_temp-np.min(img_temp))/(np.max(img_temp) - np.min(img_temp))
    img_temp_norm = (img_temp-np.min(img_temp))/(np.max(img_temp) - np.min(img_temp))


    #Original root function
    #return -np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.))) * (abs(pix_val/maxval)**(1./power))
    return -np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.))) * sigmoid_function(img_temp_norm, a, b)


# FUNCTION TO MASK
def motion_mask_parallel(img_temp, maxval, val999, width, power):
    '''Parallel computation of mask
    '''
   
    y_array = []
    for x in range(len(img_temp)):
        y_array.append(np.arange(0,len(img_temp), 1))
        
    y_array = np.vstack(y_array)
    motion_mask = img_temp*mangle(width, np.abs(64-y_array), img_temp, maxval, power, val999)
    
                   
    motion_mask = (motion_mask-np.min(motion_mask))/(np.max(motion_mask)-np.min(motion_mask))

    idx = np.where(motion_mask==0)
    motion_mask[idx]=1
    
    #motion_mask = motion_mask * img_temp
        
    return motion_mask

def save_npz_data(names, 
                  lockout_window,
                  n_sec_window,
                  feature_name,
                  selected_sessions_animal, 
                  best_sessions_animal):

    #feature_name = 'code_04'
    #lockout_window = 10
    #n_sec_window = 10

    for name in names:

        # 
        fname_out = '/home/cat/'+name+'.npz'

        # 
        data_04_list = []
        data_04_random_list = []
        data_04_lockout_list = []
        data_04_lockout_random_list = []

        session_list = []
        ctr_list = []


        root_dir = '/media/cat/4TBSSD/yuki/'+name

        temp_recs = np.load(root_dir+'/tif_files.npy')
        recordings =[]
        for k in range(len(temp_recs)):
            try:
                recordings.append(str(os.path.split(temp_recs[k])[1][:-4], "utf-8"))
            except:
                recordings.append(os.path.split(temp_recs[k])[1][:-4])

        print ("PROCESSING: ", name)

        print ("rec id,      rec name,           all rewarded trials,   "+
               str(n_sec_window) + " sec lockout rewarded trials (*** good sessions; ####### best 3 sessions")
        for ctr,recording in enumerate(recordings):

            try: 
                (data_04, data_04_random, data_04_lockout, data_04_lockout_random) = load_trial_courses_ROI_code04_trigger(
                                                                                                      recording,
                                                                                                      root_dir,
                                                                                                      feature_name,
                                                                                                      lockout_window,
                                                                                                      n_sec_window)
                data_04_list.append(data_04)
                data_04_random_list.append(data_04_random)
                data_04_lockout_list.append(data_04_lockout)
                data_04_lockout_random_list.append(data_04_lockout_random)

                session_list.append(recording)
                ctr_list.append(ctr)

            except:
                data_04 = np.zeros((0),'float32')
                data_04_random = data_04
                data_04_lockout = data_04
                data_04_lockout_random = data_04

                data_04_list.append(data_04)
                data_04_random_list.append(data_04)
                data_04_lockout_list.append(data_04)
                data_04_lockout_random_list.append(data_04)

                session_list.append(recording)
                ctr_list.append(ctr)

            # MAKE PRINTOUT TABLE
            prefix = '       '
            if ctr in selected_sessions_animal:
                if ctr in best_sessions_animal:
                    prefix="#######"
                else:
                    prefix='    ***'

            print (prefix,ctr, "     ", recording,"    ", data_04.shape, "        ", data_04_lockout.shape)

        np.savez(fname_out,
                 data_04 = data_04_list, 
                 data_04_random = data_04_random_list, 
                 data_04_lockout = data_04_lockout_list,
                 data_04_lockout_random= data_04_lockout_random_list,
                 session_list = session_list, 
                 ctr_list = ctr_list,
                 selected_sessions = selected_sessions_animal,
                 best_sessions = best_sessions_animal)

In [4]:
##################################################################
##### COMPUTE ROI TRIAL COURSES - WITH & WITHOUT LOCKOUT #########
##################################################################
# GCaMP6s: IA1, IA2
# 
names = ['IA1','IA2','IA3','IJ1','IJ2','AR4','AQ2']
#names = ['AQ2']
#names = ['IJ1','IJ2','AQ2']

from tqdm import tqdm
feature_name = 'code_04'
midline_filter_flag = False

# window to compute
n_sec_window = 10
lockout_window = n_sec_window

recompute = False  # overwrite previously generated data

for name in names:
    root_dir = '/media/cat/4TBSSD/yuki/'+name
    #recordings = np.loadtxt(root_dir + '/'+name+'.txt',dtype='str')
    
    temp_recs = np.load(root_dir+'/tif_files.npy')
    recordings =[]
    for k in range(len(temp_recs)):
        try:
            recordings.append(str(os.path.split(temp_recs[k])[1][:-4], "utf-8"))
        except:
            recordings.append(os.path.split(temp_recs[k])[1][:-4])
    
    print ("PROCESSING: ", name)
    
    if False:
        import parmap
        res = parmap.map(compute_trial_courses_ROI_code04_trigger, 
                       recordings,
                       root_dir,
                       feature_name,
                       lockout_window,
                       n_sec_window,
                       recompute,
                       midline_filter_flag,
                       pm_processes=2,
                       pm_pbar=True)
    else:
        res = []
        for recording in tqdm(recordings):
            print ("recording: ", recording)
            res.append(compute_trial_courses_ROI_code04_trigger(recording,
                                                                root_dir,
                                                                feature_name,
                                                                lockout_window,
                                                                n_sec_window,
                                                                recompute,
                                                                midline_filter_flag))
    print ("DONE WITH ", name)
    print ('')
    print ('')
    print ('')


  0%|          | 0/44 [00:00<?, ?it/s]

PROCESSING:  IJ1
recording:  IJ1pm_Feb1_30Hz
recording:  IJ1pm_Feb2_30Hz
recording:  IJ1pm_Feb3_30Hz


  7%|▋         | 3/44 [00:35<08:05, 11.83s/it]

recording:  IJ1pm_Feb4_30Hz


  9%|▉         | 4/44 [01:14<13:25, 20.13s/it]

...

recording:  AQ2am_Apr28_Week8_30Hz


 99%|█████████▉| 109/110 [1:22:16<01:10, 70.97s/it]

recording:  AQ2am_Apr29_Week8_30Hz


100%|██████████| 110/110 [1:23:18<00:00, 45.44s/it]

DONE WITH  AQ2





In [10]:
#########################################################
############## MAKE ORDERED .NZP FILES ##################
#########################################################

# MANUALLY FIND WHICH SESSIONS HAVE LARGE DIFFERENCE in BEWEEN WITH /WITHOUT PULL OVERLAP IN 10SEC WINDOW
# AQ2
selected_sessions_AQ2 = [32,50,53,54,56,57,58,59,63,64,65,66,67,70,71,72,73,74,77,78,79,80,81]
selected_sessions_AQ2 = np.append(selected_sessions_AQ2,np.arange(83,110))
best_sessions_AQ2 = [32,58,91]

# IA1
selected_sessions_IA1 = [2,10,11,13,15,29]
best_sessions_IA1 = [13,15,29]

# IA2
selected_sessions_IA2 = [3,4,13,21,28,29,30,31,32,35,40,43]
best_sessions_IA2 = [3,29,40]

# IA3
selected_sessions_IA3 = [1,2,7,17,21,25,31,32,33,34,35,36,37,38,39,40,41,42,43]
best_sessions_IA3 = [2,34,43]

# IJ1
selected_sessions_IJ1 = [6,7,8,10,14,18,20,21,22,38]
best_sessions_IJ1 = [14,22,38]

# IJ2
selected_sessions_IJ2 = [2,17,18,19,20,26,33,37,39,40,43]
best_sessions_IJ2 = [20,33,40]

# AR4
selected_sessions_AR4 = [6,12,14,23,26,27,28,30,31,32]
best_sessions_AR4 = [6,23,30]


####################################################
# SELECT A RECORDING TO COMBINE
# CHECK ALL VS. LOCKOUT DATA
#names = ['IA1','IA2','IA3','IJ1','IJ2','AR4','AQ2']
names = ['AQ2']

selected_sessions_animal = selected_sessions_AQ2
best_sessions_animal = best_sessions_AQ2

save_npz_data(names, 
              lockout_window,
              n_sec_window,
              feature_name,
              selected_sessions_animal, 
              best_sessions_animal)


PROCESSING:  AQ2
rec id,      rec name,           all rewarded trials,   15 sec lockout rewarded trials (*** good sessions; ####### best 3 sessions
        0       AQ2am_Dec9_30Hz      (0,)          (0,)
        1       AQ2am_Dec10_30Hz      (0,)          (0,)
        2       AQ2pm_Dec10_30Hz      (0,)          (0,)
        3       AQ2am_Dec11_30Hz      (7, 35, 901)          (7, 35, 901)
        4       AQ2pm_Dec14_30Hz      (10, 35, 901)          (8, 35, 901)
        5       AQ2am_Dec14_30Hz      (0,)          (0,)
        6       AQ2pm_Dec16_30Hz      (27, 35, 901)          (20, 35, 901)
        7       AQ2am_Dec17_30Hz      (21, 35, 901)          (12, 35, 901)
        8       AQ2pm_Dec17_30Hz      (5, 35, 901)          (5, 35, 901)
        9       AQ2am_Dec18_30Hz      (7, 35, 901)          (7, 35, 901)
        10       AQ2pm_Dec18_30Hz      (4, 35, 901)          (4, 35, 901)
        11       AQ2am_Dec21_30Hz      (2, 35, 901)          (2, 35, 901)
        12       AQ2am_Dec22_30Hz 

In [32]:
# # CHECK ALL VS. LOCKOUT DATA
# names = ['IA1','IA2','IA3','IJ1','IJ2','AR4','AQ2']
# names = ['AQ2']

# selected_sessions = [3, 30, 50]

# from tqdm import tqdm
# feature_name = 'code_04'
# lockout_window = 10
# n_sec_window = 10

# for name in names:
#     root_dir = '/media/cat/4TBSSD/yuki/'+name
#     #recordings = np.loadtxt(root_dir + '/'+name+'.txt',dtype='str')
    
#     temp_recs = np.load(root_dir+'/tif_files.npy')
#     recordings =[]
#     for k in range(len(temp_recs)):
#         try:
#             recordings.append(str(os.path.split(temp_recs[k])[1][:-4], "utf-8"))
#         except:
#             recordings.append(os.path.split(temp_recs[k])[1][:-4])
    
#     print ("PROCESSING: ", name)
    
#     print ("rec id,      rec name,           all rewarded trials,   10sec lockout rewarded trials")
#     for ctr,recording in enumerate(recordings):

#         try: 
#             (data_04, data_04_random, data_04_lockout, data_04_lockout_random) = load_trial_courses_ROI_code04_trigger(
#                                                                       recording,
#                                                                       root_dir,
#                                                                       feature_name,
#                                                                       lockout_window,
#                                                                       n_sec_window)
#             if ctr not in selected_sessions:
#                 print ("   ",ctr, "     ", recording,"    ", data_04.shape, "        ", data_04_lockout.shape)
#             else:
#                 print ("***", ctr, "     ", recording,"    ", data_04.shape, "        ", data_04_lockout.shape)
            
#             #print ('')
#         except:
#             pass



PROCESSING:  AQ2
rec id,      rec name,           all rewarded trials,   10sec lockout rewarded trials
*** 3       AQ2am_Dec11_30Hz      (7, 35, 601)          (7, 35, 601)
    4       AQ2pm_Dec14_30Hz      (10, 35, 601)          (9, 35, 601)
    6       AQ2pm_Dec16_30Hz      (27, 35, 601)          (22, 35, 601)
    7       AQ2am_Dec17_30Hz      (21, 35, 601)          (15, 35, 601)
    8       AQ2pm_Dec17_30Hz      (5, 35, 601)          (5, 35, 601)
    9       AQ2am_Dec18_30Hz      (7, 35, 601)          (7, 35, 601)
    10       AQ2pm_Dec18_30Hz      (5, 35, 601)          (5, 35, 601)
    11       AQ2am_Dec21_30Hz      (2, 35, 601)          (2, 35, 601)
    14       AQ2am_Dec28_30Hz      (24, 35, 601)          (13, 35, 601)
    15       AQ2am_Dec29_30Hz      (33, 35, 601)          (27, 35, 601)
    17       AQ2am_Dec31_30Hz      (65, 35, 601)          (51, 35, 601)
    18       AQ2am_Jan4_30Hz      (52, 35, 601)          (42, 35, 601)
    19       AQ2am_Jan5_30Hz      (52, 35, 601)    

In [ ]:
# # MAKE ORDERED .NZP FILES
# # MUST INPUT a temporally ordered.txt file
# fname_list = '/media/cat/4TBSSD/yuki/time_courses/AQ2_ordered.txt'
# fnames = np.loadtxt(fname_list,dtype='str')
# #print (fnames)

# data_list = []
# fnames_data_list = []
# data_random_list = []
# fnames_data_random_list = []
# for fname in fnames:
#     if '04' in fname:
#         data_list.append(np.load(fname))
#         fnames_data_list.append(fname)
#     else:
#         data_random_list.append(np.load(fname))
#         fnames_data_random_list.append(fname)

# np.savez(fname_list[:-4]+'.npz',
#         data_04_lever_pull = data_list,
#         data_04_lever_pull_fnames = fnames_data_list,
#         data_random = data_random_list,
#         data_random_fnames = fnames_data_random_list)


In [ ]:
# GET DISTRIBUTION OF SESSIONS IN EACH ANIMAL

all_files = [
'/media/cat/4TBSSD/yuki/time_courses/AR4_ordered.npz',
'/media/cat/4TBSSD/yuki/time_courses/IA1_ordered.npz',
'/media/cat/4TBSSD/yuki/time_courses/IA2_ordered.npz',
'/media/cat/4TBSSD/yuki/time_courses/IA3_ordered.npz',
'/media/cat/4TBSSD/yuki/time_courses/IJ1_ordered.npz',
'/media/cat/4TBSSD/yuki/time_courses/IJ2_ordered.npz',
'/media/cat/4TBSSD/yuki/time_courses/AQ2_ordered.npz'
]

tot = 0
for ctr, file_ in enumerate(all_files):
    temp = np.load(file_, allow_pickle=True)
    #fnames = temp['data_04_lever_pull_fnames']
    #print (fnames)
    data = temp['data_04_lever_pull']
    print (file_, " # of sessions: ", len(data))
    tot+=len(data)
    ax=plt.subplot(2,4,ctr+1)
    plt.title(os.path.split(file_)[1])
    
    lengths = []
    for k in range(len(data)):
        lengths.append(len(data[k]))
        #print (data[k].shape)
    y = np.histogram(lengths, np.arange(0,400,10))
    plt.plot(y[1][:-1],y[0])
    plt.ylabel("# of sessions")
    plt.xlabel("# of trials in sessions")
    plt.xlim(0,200)
print ("total: ", tot)

In [ ]:
# SAVE DATA
#np.save(root_dir+"data_random_code_"+str(locs_selected.shape[0])+"trials_.npy", data_stm)

In [66]:
data = np.load('/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Apr6_30Hz/data_random_code_64trials_ROIs.npy')
print (data.shape)

(64, 35, 601)


In [10]:
data= np.load('/media/cat/4TBSSD/yuki/AR4/tif_files.npy')
print (data)

['/media/cat/12TB/in_vivo/tim/yuki/AR4/tif_files/AR4_Oct27pm_15Hz_8x8/AR4_Oct27pm_15Hz_8x8.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/AR4/tif_files/AR4_Oct28pm_15Hz_8x8/AR4_Oct28pm_15Hz_8x8.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/AR4/tif_files/AR4_Oct29pm_15Hz_8x8/AR4_Oct29pm_15Hz_8x8.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/AR4/tif_files/AR4_Nov3pm_15Hz_8x8/AR4_Nov3pm_15Hz_8x8.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/AR4/tif_files/AR4_Nov3am_15Hz_8x8/AR4_Nov3am_15Hz_8x8.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/AR4/tif_files/AR4_Nov4am_15Hz_8x8/AR4_Nov4am_15Hz_8x8.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/AR4/tif_files/AR4_Nov5am_15Hz_8x8/AR4_Nov5am_15Hz_8x8.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/AR4/tif_files/AR4_Nov6am_15Hz_8x8/AR4_Nov6am_15Hz_8x8.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/AR4/tif_files/AR4_Nov7am_15Hz_8x8/AR4_Nov7am_15Hz_8x8.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/AR4/tif_files/AR4_Nov10am_15Hz_8x8/AR4_Nov10am_15Hz_8x8.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/AR4/tif_files/AR4

In [30]:
print (np.load('/media/cat/4TBSSD/yuki/IA1/tif_files.npy'))

['/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb1_30Hz/IA1pm_Feb1_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb2_30Hz/IA1pm_Feb2_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb4_30Hz/IA1pm_Feb4_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb5_30Hz/IA1pm_Feb5_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb9_30Hz/IA1pm_Feb9_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb10_30Hz/IA1pm_Feb10_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb11_30Hz/IA1pm_Feb11_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb12_30Hz/IA1pm_Feb12_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb15_30Hz/IA1pm_Feb15_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb16_30Hz/IA1pm_Feb16_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_f

In [17]:
import h5py
filename = "/home/cat/Downloads/AR4_2014-12-02_12-56-34.975DLC_resnet50_yuki_leverJul21shuffle1_200000.h5"

f = h5py.File(filename, "r")
# List all groups
print("Keys: %s" % f.keys())
a_group_key = list(f.keys())[0]

# Get the data
data = list(f[a_group_key])

data2 = f['df_with_missing']['table']

Keys: <KeysViewHDF5 ['df_with_missing']>


In [31]:
print (data2[0][1].shape)

traces = []
for k in range(len(data2)):
    traces.append(data2[k][1].reshape(-1,3))
    

(21,)


In [33]:
traces = np.array(traces)
print (traces.shape)

(20066, 7, 3)


In [11]:
files = np.load('/media/cat/4TBSSD/yuki/IJ1/tif_files.npy')
print (files[0].decode("utf-8"))

AttributeError: 'str' object has no attribute 'decode'